In [2]:
!pip install nlpaug
!pip install nltk

In [3]:
import pandas as pd
final_df = pd.read_csv('/kaggle/input/last-input-for-ml-dl/final_train_with_stopwords.csv') 
dev = pd.read_csv('/kaggle/input/last-input-for-ml-dl/final_dev_with_stopwords (1).csv')
# test = pd.read_csv('/kaggle/input/final-dataset-with-stop-words/final_test_with_stopwords.csv',delimiter=',')
test_label = pd.read_csv('/kaggle/input/last-input-for-ml-dl/final_test_with_stopwords.csv')

In [4]:
final_df.duplicated().sum()

132

In [5]:
final_df['text'].duplicated().sum()

206

In [6]:
final_df.drop_duplicates(subset='text',inplace=True)

In [7]:
final_df.duplicated().sum()

0

In [8]:
dev.duplicated().sum()

10

In [9]:
dev['text'].duplicated().sum()

12

In [10]:
dev.drop_duplicates(subset='text',inplace=True)

In [11]:
test_label['text'].duplicated().sum()

15

In [12]:
final_df['label'].value_counts()

label
not_applicable    3600
hope              1820
hate              1264
Name: count, dtype: int64

In [13]:
dev['label'].value_counts()

label
not_applicable    799
hope              404
hate              261
Name: count, dtype: int64

In [14]:
dev['text'].isna().sum()

0

In [15]:
dev.duplicated().sum()

0

In [16]:
dev.head()

,text,label
0,اخترتك حبا لقلبي فلا تكن بغيابك له وجعا كنت وم...,hope
1,ضعي ما تشاءين من مساحيق التجميل فانا اتلذذ في ...,not_applicable
2,ان شعوري بالغضب والرغبه في المضي قدما في اشياء...,hope
3,بل ايران جندت نفسها للشيطان ومشروعها الطاءفي و...,not_applicable
4,الحب ياتي مره واحده في العمر او قد لا ياتي ابد...,hope


In [17]:
x = final_df['text'].apply(lambda x: len(str(x).split(" ")))
y = dev['text'].apply(lambda x: len(str(x).split(" ")))
z = test_label['text'].apply(lambda x: len(str(x).split(" ")))
print(x.min(),x.max(),y.min(),y.max(),z.min(),z.max())

1 130 1 101 1 112


In [18]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1
    }

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

# === Step 1: Encode Labels ===
le = LabelEncoder()
final_df['label_enc'] = le.fit_transform(final_df['label'])
dev['label_enc'] = le.fit_transform(dev['label'])
# Map of encoded labels to original:
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42, stratify=final_df['label_enc'])
train_df = final_df
val_df = dev

# === Step 2: Prepare Tokenizer & Model ===
model_name = "aubmindlab/bert-large-arabertv02" 
# model_name = "UBC-NLP/MARBERT"
# model_name = "asafaya/bert-base-arabic"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))

# dropout rates
model.config.hidden_dropout_prob = 0.2
model.config.attention_probs_dropout_prob = 0.2

# === Step 3: Dataset Class ===
class EmotionDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=256)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# === Step 4: Prepare Datasets ===
train_dataset = EmotionDataset(train_df['text'].tolist(), train_df['label_enc'].tolist())
val_dataset = EmotionDataset(val_df['text'].tolist(), val_df['label_enc'].tolist())
# For dev dataset, since no labels, just encode texts only
dev_dataset = EmotionDataset(test_label['text'].tolist())

# === Step 5: Training Arguments ===

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    disable_tqdm=False,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    # greater_is_better=True,
    report_to="none",
    learning_rate=3.5e-6,
    weight_decay=0.01,  
    warmup_ratio=0.1, 
    # gradient_accumulation_steps=2,  
)


# === Step 6: Trainer Initialization ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)] 
)

# === Step 7: Train ===
trainer.train()


Label mapping: {'hate': 0, 'hope': 1, 'not_applicable': 2}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-large-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.036800,1.032201,0.505464,0.432336
2,1.003700,0.944895,0.553279,0.452039
3,0.879200,0.837504,0.584016,0.524553
4,0.743800,0.746317,0.647541,0.637644
5,0.762700,0.719422,0.661202,0.655732
6,0.674100,0.710900,0.662568,0.658225
7,0.647800,0.709959,0.670765,0.667545
8,0.682200,0.709258,0.664617,0.662323
9,0.632900,0.705459,0.668716,0.666076
10,0.598900,0.702428,0.675546,0.671950


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=5852, training_loss=0.7497044154738859, metrics={'train_runtime': 7493.6674, 'train_samples_per_second': 17.839, 'train_steps_per_second': 1.116, 'total_flos': 2.844437412588005e+16, 'train_loss': 0.7497044154738859, 'epoch': 14.0})

In [27]:
eval_results = trainer.evaluate()
print(eval_results)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.7024281620979309, 'eval_accuracy': 0.6755464480874317, 'eval_f1': 0.6719501002930802, 'eval_runtime': 34.3573, 'eval_samples_per_second': 42.611, 'eval_steps_per_second': 2.678, 'epoch': 14.0}


In [28]:
# === Step 8: Predict on dev dataset ===
predictions = trainer.predict(dev_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
pred_labels = le.inverse_transform(preds)

# === Step 9: Add predictions to dev and save ===
# test['prediction'] = pred_labels
# print(test[['id', 'prediction']].head())
# test = test[['id', 'prediction']]

# test.to_csv('prediction.csv', index=False)

In [22]:
# test

In [23]:
# test['prediction'].value_counts()

In [24]:
# import zipfile
# with zipfile.ZipFile("prediction.zip", 'w', zipfile.ZIP_DEFLATED) as zipf:
#     zipf.write("prediction.csv")

In [29]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

y_true = test_label['label']
y_pred = pred_labels

# Labels in a fixed order
labels = ["hate", "hope", "not_applicable"]

# 1. Full report (per class + macro avg)
print(classification_report(y_true, y_pred, labels=labels, digits=3))

# 2. Macro precision, recall, f1 only
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=labels, average="macro"
)
print(f"Macro Precision: {precision:.3f}")
print(f"Macro Recall:    {recall:.3f}")
print(f"Macro F1-score:  {f1:.3f}")


                precision    recall  f1-score   support

          hate      0.684     0.672     0.678       287
          hope      0.697     0.616     0.654       422
not_applicable      0.697     0.746     0.721       768

      accuracy                          0.695      1477
     macro avg      0.693     0.678     0.684      1477
  weighted avg      0.695     0.695     0.693      1477

Macro Precision: 0.693
Macro Recall:    0.678
Macro F1-score:  0.684
